# Dependencies

pip install luma.oled luma.core pillow pyftdi


# FTDI Device Info

In [1]:
from pyftdi.ftdi import Ftdi

devices = Ftdi.list_devices()

if not devices:
    print("❌ No FTDI devices found.")
else:
    for entry in devices:
        try:
            # entry can be (UsbDeviceDescriptor, interface) or similar
            if isinstance(entry, tuple):
                dev_desc = entry[0]
                interfaces = entry[1] if len(entry) > 1 else []
            else:
                dev_desc = entry
                interfaces = []

            # UsbDeviceDescriptor fields
            vendor = getattr(dev_desc, "vid", None)
            product = getattr(dev_desc, "pid", None)
            serial = getattr(dev_desc, "sn", None)
            description = getattr(dev_desc, "description", None)

            print("✅ FTDI Device Found:")
            print(f"   Vendor ID   : {vendor:04x}" if vendor else "   Vendor ID   : Unknown")
            print(f"   Product ID  : {product:04x}" if product else "   Product ID  : Unknown")
            print(f"   Serial No.  : {serial or 'Unknown'}")
            print(f"   Description : {description or 'Unknown'}")
            print(f"   Interfaces  : {interfaces}")
            print()

        except Exception as e:
            print(f"⚠️ Could not parse entry {entry}: {e}")


✅ FTDI Device Found:
   Vendor ID   : 0403
   Product ID  : 6014
   Serial No.  : ￿￿￿￿￿￿￿￿
   Description : ￿￿￿￿￿￿
   Interfaces  : 1



In [2]:
from pyftdi.ftdi import Ftdi

def safe_to_hex_string(value):
    """Return UTF-8 string if printable, else show hex representation."""
    if value is None:
        return 'None'
    try:
        if all(0x20 <= ord(c) <= 0x7E for c in value):
            return value  # printable ASCII
        else:
            raise UnicodeEncodeError
    except Exception:
        return ' '.join(f'{ord(c):02X}' for c in value)

devices = Ftdi.list_devices()

if not devices:
    print("❌ No FTDI devices found.")
else:
    for entry in devices:
        try:
            if isinstance(entry, tuple):
                dev_desc = entry[0]
                interfaces = entry[1] if len(entry) > 1 else []
            else:
                dev_desc = entry
                interfaces = []

            vendor = getattr(dev_desc, "vid", None)
            product = getattr(dev_desc, "pid", None)
            serial = getattr(dev_desc, "sn", None)
            description = getattr(dev_desc, "description", None)

            print("✅ FTDI Device Found:")
            print(f"   Vendor ID   : {vendor:04X}" if vendor else "   Vendor ID   : Unknown")
            print(f"   Product ID  : {product:04X}" if product else "   Product ID  : Unknown")
            print(f"   Serial No.  : 0x{safe_to_hex_string(serial)}")
            print(f"   Description : 0x{safe_to_hex_string(description)}")
            print(f"   Interfaces  : {interfaces}")
            print()

        except Exception as e:
            print(f"⚠️ Could not parse entry {entry}: {e}")


✅ FTDI Device Found:
   Vendor ID   : 0403
   Product ID  : 6014
   Serial No.  : 0xFFFF FFFF FFFF FFFF FFFF FFFF FFFF FFFF
   Description : 0xFFFF FFFF FFFF FFFF FFFF FFFF
   Interfaces  : 1



# FTDI FT232H Sanity Test

In [3]:
from pyftdi.ftdi import Ftdi
from pyftdi.i2c import I2cController
from IPython.display import display, Markdown
import traceback

'''
Bus Rate
-  100 kHz ... default
-  400 kHz
- 1000 kHz
'''

BUS_RATE = 4e5


display(Markdown("## 🔍 FT232H Sanity Test"))

try:
    display(Markdown("**🔹 Scanning for FTDI devices...**"))
    Ftdi.show_devices()
except Exception as e:
    display(Markdown(f"❌ **Could not list FTDI devices:** `{e}`"))
    traceback.print_exc()
else:
    try:
        display(Markdown("**🔹 Initializing I2C interface...**"))
        i2c = I2cController()
        i2c.configure('ftdi://ftdi:232h/1', frequency=BUS_RATE) 


        freq = i2c.frequency
        display(Markdown(f"✅ **Connected to FT232H via I2C** at **{freq/1000:.1f} kHz**"))

        # ✅ Manually scan I2C addresses 0x03–0x77
        display(Markdown("**🔹 Scanning I2C bus...**"))
        found = []
        for addr in range(0x03, 0x78):
            try:
                port = i2c.get_port(addr)
                port.read(1)  # Try reading one byte (safe test)
                found.append(addr)
            except Exception:
                # No device responded
                pass

        if found:
            md = "\n".join([f"- `0x{addr:02X}`" for addr in found])
            display(Markdown(f"✅ **Found I2C devices:**\n{md}"))
        else:
            display(Markdown("⚠️ **No I2C devices detected.** Check wiring or power."))

        i2c.terminate()
        display(Markdown("✅ **Test complete — FT232H is working properly.**"))

    except Exception as e:
        display(Markdown(f"❌ **Failed to communicate with FT232H:** `{e}`"))
        traceback.print_exc()


## 🔍 FT232H Sanity Test

**🔹 Scanning for FTDI devices...**

Available interfaces:
  ftdi://ftdi:232h:1/1  (￿￿￿￿￿￿)



**🔹 Initializing I2C interface...**

✅ **Connected to FT232H via I2C** at **400.0 kHz**

**🔹 Scanning I2C bus...**

✅ **Found I2C devices:**
- `0x3C`

✅ **Test complete — FT232H is working properly.**

# SH1106 Oled Display 128x64

In [17]:
from pyftdi.i2c import I2cController

# Connect FT232H
i2c = I2cController()
i2c.configure('ftdi://ftdi:232h/1')
oled = i2c.get_port(0x3C)

# Example: write a command
oled.write([0x00, 0xAE])  # Display OFF
oled.write([0x00, 0xAF])  # Display ON

In [ ]:
oled.write([0x00, 0xAE])  # Display OFF

In [18]:
oled.write([0x00, 0xAF])  # Display ON

In [20]:
i2c.terminate()

# Luma Display Library

In [2]:
from pyftdi.i2c import I2cController
from luma.oled.device import sh1106
from luma.core.interface.serial import i2c
from PIL import Image, ImageDraw, ImageFont

# ---- Adapter to make PyFTDI compatible with luma ----
class SMBusAdapter:
    def __init__(self, port):
        self._port = port
    def write_i2c_block_data(self, i2c_addr, register, data):
        self._port.write([register] + list(data))
    def write_byte_data(self, i2c_addr, register, value):
        self._port.write([register, value])
# -----------------------------------------------------

# Initialize FT232H
ctrl = I2cController()
ctrl.configure('ftdi://::/1', frequency=400000)     
port = ctrl.get_port(0x3C)       

# Make luma-compatible bus
smbus_like = SMBusAdapter(port)
serial = i2c(bus=smbus_like, address=0x3C)
display = sh1106(serial)

# ---- Manual drawing mode ----
# Create a blank 1-bit image the same size as the display
image = Image.new("1", (display.width, display.height))
draw = ImageDraw.Draw(image)
font = ImageFont.load_default()

# Draw text
draw.text((0, 0), "Hello FT232H!", font=font, fill=255)
draw.text((0, 16), "SH1106 OLED", font=font, fill=255)

# Send image buffer to OLED
display.display(image)


In [77]:
# --- Terminate cleanly ---
display.cleanup()   # optional: clear display & release framebuffer
ctrl.close()        # VERY IMPORTANT: release FT232H

In [40]:
draw.rectangle((0, 0, display.width, display.height), outline=0, fill=0)
draw.text((0,0), "XXX", font=font, fill=255)

In [41]:
display.display(image)

In [35]:
display.clear()

In [73]:
import time

for i in range(10):
    draw.rectangle((0, 0, display.width, display.height), outline=0, fill=0)
    display.display(image)
    draw.text((i*5,i*5), "XXX", font=font, fill=255)
    #display.command(0xAE) # display off
    display.display(image)
    #display.command(0xAF) # display on
    time.sleep(0.1)
print("done")

done


In [67]:
draw.rectangle((0, 0, display.width, display.height), outline=0, fill=0)
draw.text((0,0), "Hello, world!", font=font, fill=1)
draw.text((0,16), "I = 0.120 A", font=font, fill=1)
display.display(image)

In [32]:
from PIL import Image

# Make a blank 1-bit (black) image
blank = Image.new("1", (display.width, display.height))
display.display(blank)


In [ ]:
import time

for i in range(10):
    draw.rectangle((0, 0, display.width, display.height), outline=0, fill=0)
    display.display(image)
    time.sleep(0.1)
    draw.rectangle((0, 0, display.width, display.height), outline=0, fill=1)
    display.display(image)
    #draw.text((i*5,i*5), "XXX", font=font, fill=255)
    #display.command(0xAE) # display off
    #display.display(image)
    #display.command(0xAF) # display on
    time.sleep(0.1)
print("done")